### Example Code

Our sample code will be the same algorithm we've used in the other examples. In Julia, this is implemented by

In [ ]:
function f(a,b)
    x = a
    if a>b
        x+=20
    else
        x+=b
    end
    return x
end

### Lexical Analysis

We will use the `Tokenize` package to analyze our code. We can install this package like so:

In [ ]:
using Pkg
Pkg.add("Tokenize")

Then we can use this code to see the token stream outputted by lexical analysis. Try it for yourself!


In [1]:
using Tokenize

collect(tokenize("function f(a,b)
    x = a
    if a>b
        x+=20
    else
        x+=b
    end
    return x
end"))

39-element Vector{Tokenize.Tokens.Token}:
 1,1-1,8          KEYWORD        "function"
 1,9-1,9          WHITESPACE     " "
 1,10-1,10        IDENTIFIER     "f"
 1,11-1,11        LPAREN         "("
 1,12-1,12        IDENTIFIER     "a"
 1,13-1,13        COMMA          ","
 1,14-1,14        IDENTIFIER     "b"
 1,15-1,15        RPAREN         ")"
 1,16-2,4         WHITESPACE     "\n    "
 2,5-2,5          IDENTIFIER     "x"
 2,6-2,6          WHITESPACE     " "
 2,7-2,7          OP             "="
 2,8-2,8          WHITESPACE     " "
 ⋮
 6,9-6,9          IDENTIFIER     "x"
 6,10-6,11        OP             "+="
 6,12-6,12        IDENTIFIER     "b"
 6,13-7,4         WHITESPACE     "\n    "
 7,5-7,7          KEYWORD        "end"
 7,8-8,4          WHITESPACE     "\n    "
 8,5-8,10         KEYWORD        "return"
 8,11-8,11        WHITESPACE     " "
 8,12-8,12        IDENTIFIER     "x"
 8,13-9,0         WHITESPACE     "\n"
 9,1-9,3          KEYWORD        "end"
 9,4-9,3          ENDMARKER      "

### Abstract Syntax Tree (AST)

To create the AST of our function we simply use the dump function:

In [ ]:
dump(:(function f(a,b)
    x = a
    if a>b
        x+=20
    else
        x+=b
    end
    return x
end))

Since this is a long output, it is attached here for convenience
``` julia
Expr
  head: Symbol function
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol f
        2: Symbol a
        3: Symbol b
    2: Expr
      head: Symbol block
      args: Array{Any}((7,))
        1: LineNumberNode
          line: Int64 1
          file: Symbol In[20]
        2: LineNumberNode
          line: Int64 2
          file: Symbol In[20]
        3: Expr
          head: Symbol =
          args: Array{Any}((2,))
            1: Symbol x
            2: Symbol a
        4: LineNumberNode
          line: Int64 3
          file: Symbol In[20]
        5: Expr
          head: Symbol if
          args: Array{Any}((3,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol >
                2: Symbol a
                3: Symbol b
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr
            3: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr
        6: LineNumberNode
          line: Int64 8
          file: Symbol In[20]
        7: Expr
          head: Symbol return
          args: Array{Any}((1,))
            1: Symbol x
```
As we can see, Julia's AST is long but quite intuitive to read. For more information, you can use this link: [Julia ASTs](https://julia-doc.readthedocs.io/en/latest/devdocs/ast/).

### Intermediate Representation

The next step is creating the LLVM IR. We can print this out pretty easily

In [4]:
@code_llvm f(3,4)

;  @ In[3]:1 within `f`
; Function Attrs: uwtable
define i64 @julia_f_1485(i64 signext %0, i64 signext %1) #0 {
top:
;  @ In[3]:3 within `f`
; ┌ @ operators.jl:378 within `>`
; │┌ @ int.jl:83 within `<`
    %.not = icmp slt i64 %1, %0
; └└
  %value_phi.v = select i1 %.not, i64 20, i64 %1
  %value_phi = add i64 %value_phi.v, %0
;  @ In[3]:8 within `f`
  ret i64 %value_phi
}


One thing to note is that for this to work, you need to run the example code once and give the program some concrete input to work through. Once you run it, however, the Julia IR is a little bit more optimized and easier to read compared to the other languages LLVM IR results.
